<a href="https://colab.research.google.com/github/ppdevexe/ml-projects/blob/master/sturctured_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  sklearn
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of targets:', label_batch )
example_batch = next(iter(train_ds))[0]
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())
age = feature_column.numeric_column("age")
demo(age)
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

193 train examples
49 validation examples
61 test examples
Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([59 45 66 42 51], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[41.]
 [53.]
 [68.]
 [57.]
 [62.]]

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are por

In [ ]:
crossed_feature=feature_column.crossed_column([age_buckets,thal],hash_bucket_size=1000)
demo=feature_column.indicator_column(crossed_feature)
demo0

IndicatorColumn(categorical_column=CrossedColumn(keys=(BucketizedColumn(source_column=NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(18, 25, 30, 35, 40, 45, 50, 55, 60, 65)), VocabularyListCategoricalColumn(key='thal', vocabulary_list=('fixed', 'normal', 'reversible'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), hash_bucket_size=1000, hash_key=None))

In [ ]:
feature_columns=[]
for header in['age','trestbps','chol','thalach','oldpeak','slope','ca']:
  feature_columns.append(feature_column.numeric_column(header))
age_buckets=feature_column.bucketized_column(age,boundaries=[18,20,30,35,40,45,50,55,60,65])
feature_columns.append(age_buckets)


In [ ]:

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

In [ ]:
feature_layer=tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
train_ds=df_to_dataset(train,batch_size=32)
val_ds=df_to_dataset(val,batch_size=32)
test_ds=df_to_dataset(test,batch_size=32)

In [ ]:
model=tf.keras.Sequential([feature_layer,layers.Dense(1,activation='sigmoid')])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'],run_eagerly=True)
model.fit(train_ds,validation_data=val_ds,epochs=5)
loss,accuracy=model.evaluate(test_ds)
print("accuracy",accuracy)

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

7/7 [==============================] - 0s 27ms/step - loss: 3.9559 - accuracy: 0.7358 - val_loss: 3.3507 - val_accuracy: 0.6735
Epoch 2/5
7/7 [==============================] - 0s 24ms/step - loss: 1.6661 - accuracy: 0.7306 - val_loss: 1.9337 - val_accuracy: 0.6735
Epoch 3/5
7/7 [==============================] - 0s 25ms/step - loss: 1.0802 - accuracy: 0.7202 - val_loss: 1.0169 - val_accuracy: 0.6327
Epoch 4/5
7/7 [==============================] - 0s 26ms/step - loss: 0.9159 - accuracy: 0.590

In [ ]:
model_nn=tf.keras.Sequential([feature_layer,layers.Dense(128,activation='relu'),
                              layers.Dense(128,activation='relu'),
                              layers.Dense(1,activation='sigmoid')])
model_nn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'],run_eagerly=True)
model_nn.fit(train_ds,validation_data=val_ds,epochs=5)


Epoch 1/5
7/7 [==============================] - 0s 32ms/step - loss: 1.5406 - accuracy: 0.5544 - val_loss: 2.2266 - val_accuracy: 0.6735
Epoch 2/5
7/7 [==============================] - 0s 31ms/step - loss: 1.0941 - accuracy: 0.6010 - val_loss: 0.6589 - val_accuracy: 0.6939
Epoch 3/5
7/7 [==============================] - 0s 30ms/step - loss: 0.5628 - accuracy: 0.7565 - val_loss: 0.4616 - val_accuracy: 0.7755
Epoch 4/5
7/7 [==============================] - 0s 30ms/step - loss: 0.5032 - accuracy: 0.7098 - val_loss: 0.8116 - val_accuracy: 0.6735
Epoch 5/5
7/7 [==============================] - 0s 34ms/step - loss: 0.5415 - accuracy: 0.7306 - val_loss: 0.7192 - val_accuracy: 0.6735


In [ ]:
print(model_nn.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_4 (DenseFeatu multiple                  24        
_________________________________________________________________
dense_4 (Dense)              multiple                  131840    
_________________________________________________________________
dense_5 (Dense)              multiple                  16512     
_________________________________________________________________
dense_6 (Dense)              multiple                  129       
Total params: 148,505
Trainable params: 148,505
Non-trainable params: 0
_________________________________________________________________
None
